# NBA Moneyline Model

Outlining my logistic regression model I built for betting on NBA games. Note: 
I am looking for value in betting, that is, not quite predicting who wins, but 
finding disparities in the odds sportsbooks give a team to win and the odds my 
model gives them to win.

#### Necessary Packages

In [ ]:
import requests
import pandas 
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from io import StringIO
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
import pandas as pd 

The requests and io libraries are used for web scraping data from basketball-reference.com and the NBA API is used to pull data from nba.com. 

#### Setup

To start, the model takes user inputs on the betting team, their opponent, whether the betting team is home or away, and the Vegas moneyline odds for the team you are betting on.

#### Building the Dataframe

##### Features

In [ ]:
featureCols = ['H1/A0', 'NetRtg', 'SRS', 'eFG%', 'OREB%', 'TOV%', 'DREB%']

The features are home/away (inputted as 1s or 0s), Net Rating, Simple Rating System, Effective FG%, Offensive Rebound %, Turnover % and Defenseive Rebound % for the opposing team.

##### Obtaining Data

The data was obtained using the following lines of code:

In [ ]:
teamID = 1610612749 #(Milwaukee Bucks)
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=str(teamID),
                                        season_type_nullable='Regular Season')
games = gamefinder.get_data_frames()[0]

In [ ]:
url = 'https://www.basketball-reference.com/leagues/NBA_2025.html'
response = requests.get(url)
teamStats = pd.read_html(StringIO(response.text), attrs={'id':'advanced-team'}, 
                         header=1)[0]

##### Creating the Dataframe to Train the Model

The above code first gathers the past 200 games with the feature variables of their opponent in each game. Then, their current opponent and their feature variables are added to the last row of the dataframe. 

For example, if I am betting on the Milwaukee Bucks at home against the Miami Heat, the dataframe the model would use would look like this (Note values here are unweighted): 

      Opponent  H1/A0  W/L  NetRtg    SRS   eFG%  OREB%  TOV%  DREB%
0        Magic      0    1    -1.8  -2.18  0.501   25.5  13.3   78.3
1        Kings      0    1     1.2   1.26  0.542   24.5  11.4   76.5
2    Mavericks      0    1     1.9   2.41  0.550   25.6  12.8   73.8
3      Rockets      0    0     4.9   5.67  0.513   31.5  11.8   75.4
4     Warriors      0    1    -0.2   0.76  0.529   26.9  12.4   76.0
..         ...    ...  ...     ...    ...    ...    ...   ...    ...
196    Blazers      0    0    -5.1  -4.62  0.521   28.1  14.1   71.9
197      Spurs      0    0    -1.6  -1.06  0.536   24.8  12.9   74.1
198  Grizzlies      0    0     7.5   6.97  0.561   29.6  13.8   74.5
199    Thunder      0    0    12.7  12.52  0.549   23.9  10.8   73.8
200       Heat      1    0     0.0  -0.71  0.540   21.5  12.4   75.6

[201 rows x 9 columns]


#### Training the Model

Using scikit-learn's train_test_split function, I trained the modeln on 90% of the data, or 180 datapoints. I turned shuffle to false because I felt that the order of the games matter to take into account the form of the team. Below, X is the feature columns of the dataframe and y is the target column, W/L

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                            shuffle=False)


I then fitted the data to the training datasets and the logistic regression was able to spit out a probability of the team winning based on its opponent's statistics. 

In [ ]:
logReg = LogisticRegression(C=0.5, solver='saga', max_iter=10**5)
logReg.fit(X_train, y_train)
yPred = logReg.predict_proba(X_test)

#### Interpreting Results

Remember, this model is looking for value against the book, not just the outright winner. Taking the sportsbook odds, I converted them into an implied probability using the following formulas:

If the odds are negative, Implied Probability = |odds|/(100+|odds|) and if the odds are positive, IP = 100/(100+odds).

Continuing the Bucks vs Heat example, this is what the model shows if the Milwaukee Bucks moneyline is set at -110:

Calculated win probability: 0.703	Book probability: 0.524
Value over book: 0.179


It can be seen here that this is a valuable bet.

#### Model Testing, Results, and Improvements

The model is currently undergoing testing and improvements. For each bet that has a positive value over book, I will bet one unit on it. After the first round of testing, in which the model was 11/33 on picks for a loss of 4.8 units, I adjusted the weight of features, especially the Home/Away feature which was too heavily accounted for in the model. Whlie scikit-learn doesn't have builtin weighing I artificially weighted features by changing their coefficients. Currently, version 2 of the model has been correct 14/33 times for a gain of 2.8 units. Over the same period, flipping a coin would have correctly guessed 18/33 for a loss of 0.5 units.